<a href="https://colab.research.google.com/github/cool1726/class2025Spring/blob/main/midterm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어처리 중간 과제: 잡코리아 합격자소서 문항-답변 크롤링
학번 2024511049  
  
### 과제 배경
- 잡코리아 합격 자소서의 문항-답변을 크롤링하는 프로젝트를 진행하였다.
- ChatGPT를 활용해 기업 채용에 쓰이는 자기소개서(이하 자소서)를 생성하고, 생성된 자소서를 데이터셋으로 하여 논문 연구를 진행하고자 한다.
- ChatGPT가 특정 산업군/직군에 대해 전문적인 자소서를 작성할 수 있도록 **고품질의 자소서 데이터셋 구축을 위해 잡코리아에서 제공하는 합격자소서를 크롤링하여 활용**하고자 한다.

### 개요
- 잡코리아 합격자소서의 경우 지원분야를 검색 필터로 제공한다. Search Filters와 Pagination은 접속 URL의 GET Parameter로 포함되므로 파라미터를 활용해 **`의료.바이오`, `마케팅.광고.MD`** 분야의 합격자소서를 크롤링한다.

### 상세 내용
- 로그인 : 잡코리아 합격자소서를 여러 건 조회하려면 로그인이 필수적이므로 chrome webdriver에서 로그인을 직접 진행한다. (잡코리아 아이디, 비밀번호를 직접 Key-in 하므로 개인정보 이슈로 블라인드 처리)
- "의료.바이오" 분야의 합격자소서 리스트를 조회할 수 있는 URL로 전체 자소서 리스트를 불러와 각 자소서별 상세 URL를 찾는다. Pagination은 Page=#NUMBER# 파라미터를 넘기는 방식으로 진행했다.
- 상세 페이지 : 상세 URL로 접속하여 자소서 문항과 답변 내용을 가져온다.
    - 자소서 첫 2개 문항만 토글되어 답변이 보이는 상태이므로 답변이 보이지 않는 문항에 대해서는 문항을 클릭하여 답변이 Unfold 되도록 액션을 추가했다.
    - 예외 처리 : 자소서별 전문가 분석이 포함된 경우가 있어 HTML 구조가 다른 경우를 대비하여 예외 처리를 적용했다. 또한 "선택하신 합격자소서 정보가 없습니다." 와 같이 유효하지 않은 URL의 경우 제외하도록 예외 처리하였다.
- Output : 합격자소서 문항-답변 쌍을 JSON 파일 형식으로 저장한다.

### 추후 방향
- `의료.바이오`, `마케팅.광고.MD` 분야의 자기소개서 크롤링을 적용하여 다양한 데이터에 대해 안정성을 검증하였으므로 추후 `AI.개발.데이터`, `제조.생산`, `엔지니어링.설계` 등 분야로 확장하여 잡코리아의 7,893건의 자소서를 모두 크롤링하는데 활용할 수 있을 것이다.

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 16.5 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import math
import csv
import random
import json

In [ ]:
login_url = "https://www.jobkorea.co.kr/Login/Login_Tot.asp?rDBName=GG&re_url=/"
bio_list_url = "https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart=10044&isSaved=1&Page="
marketing_list_url = "https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart=10030&isSaved=1&Page="

In [ ]:
# 랜덤 sleep 함수
def random_sleep(min_sec=3, max_sec=5):
    time.sleep(random.uniform(min_sec, max_sec))

In [ ]:
def login():
    options = webdriver.ChromeOptions()
    options.add_argument(" --headless")
    options.add_argument(" --disable-dev-shm-usage")
    options.add_argument(" --no-sandbox")
    driver = webdriver.Chrome(options=options)

    # 잡코리아 로그인 페이지 접속
    driver.get(login_url)

    # 로그인
    driver.find_element(By.ID, 'M_ID').send_keys('****') # id
    driver.find_element(By.ID, 'M_PWD').send_keys('****') # password
    driver.find_element(By.CLASS_NAME, 'login-button').click()
    random_sleep()
    return driver

In [ ]:
def collect_list_urls(driver, url):
    # 합격자소서 리스트 페이지 접속
    driver.get(url + "1")
    random_sleep()

    # Pagination 계산
    # 총 검색 건 수 가져오기
    total_text = driver.find_element(By.CSS_SELECTOR, '#container > div.stContainer > div:nth-child(4) > h4 > span:nth-child(2)').text
    total_count = int(''.join(filter(str.isdigit, total_text)))
    print("총 검색 결과 수:", total_count)

    # 필요한 페이지 수 계산
    per_page = 20
    total_pages = math.ceil(total_count / per_page)
    print("총 페이지 수:", total_pages)

    # 모든 페이지에서 자소서 url 수집
    essay_direct_urls = []

    for page_num in range(1, total_pages + 1):
        target_url = url + str(page_num)
        driver.get(target_url)
        random_sleep()

        elements = driver.find_elements(By.CSS_SELECTOR, "#container > div.stContainer > div.starListsWrap.ctTarget > ul > li > a")

        for elem in elements:
            href = elem.get_attribute('href')
            if href and 'View' in href:
                essay_direct_urls.append(href)

        print(f"{page_num} 페이지: 현재까지 수집된 URL 수: {len(essay_direct_urls)}")

    print("최종 수집된 URL 수:", len(essay_direct_urls))
    return essay_direct_urls

In [ ]:
def collect_essays(driver, url_lists):
    results = []

    # 각 url 별로 View 페이지 접속
    for idx, url in enumerate(url_lists, 1):
        # 문항, 답변 가져오기
        try:
            driver.get(url)
            random_sleep(2, 4)

            company = driver.find_element(By.CSS_SELECTOR, '#container > div.stContainer > div.selfTopBx > div.viewTitWrap > h2 > strong > a').text
            title = driver.find_element(By.CSS_SELECTOR, '#container > div.stContainer > div.selfTopBx > div.viewTitWrap > h2 > em').text

            questions = driver.find_elements(By.CSS_SELECTOR, 'dl.qnaLists > dt > button > span.tx')
            question_buttons = driver.find_elements(By.CSS_SELECTOR, 'dl.qnaLists > dt > button')
            # answers = driver.find_elements(By.CSS_SELECTOR, 'dl.qnaLists > dd > div.tx')

            qna_list = []

            if questions and question_buttons:
                for button, q in zip(question_buttons, questions):
                    try:
                        parent_dt = button.find_element(By.XPATH, '..')
                        if 'on' not in parent_dt.get_attribute('class'):
                            driver.execute_script("arguments[0].scrollIntoView(true);", button)
                            time.sleep(0.2)
                            button.click()
                            time.sleep(0.3)

                        parent_dd = parent_dt.find_element(By.XPATH, 'following-sibling::dd[1]')
                        answer_element = parent_dd.find_element(By.CSS_SELECTOR, 'div.tx')
                        answer_text = answer_element.text.strip()

                        qna_list.append({
                            'question': q.text.strip(),
                            'answer': answer_text
                        })
                    except Exception as e:
                        print(f"문항 클릭 실패 또는 답변 가져오기 실패: {e}")
            else:
                try:
                    questions = driver.find_elements(By.CSS_SELECTOR, 'dl.qnaLists > dt > button > span.tx')
                    answer_block = driver.find_element(By.CSS_SELECTOR, 'dl.qnaLists > dd > div.tx')

                    direct_text = answer_block.text.strip()
                    b_tags = answer_block.find_elements(By.TAG_NAME, 'b')
                    b_texts = [b.text.strip() for b in b_tags if b.text.strip()]

                    full_answer = direct_text
                    if b_texts:
                        full_answer += "\n" + "\n".join(b_texts)

                    qna_list.append({
                        'question': q.text.strip(),
                        'answer': a.text.strip()
                    })
                except Exception as e:
                    print(f"HTML 구조 오류로 가져오기 실패: {e}")

            results.append({'url': url, 'company': company, 'title': title, 'qas': qna_list})

            print(f"{idx}/{len(url_lists)} : {company} {title} 크롤링 완료")
            print(qna_list)

        except Exception as e:
            print(f"{idx}/{len(url_lists)} : {url} 에서 에러 발생: {e}")
    return results

## 의료.바이오 분야 자기소개서 수집

In [ ]:
# 잡코리아 로그인
driver = login()

# 자기소개서 목록 URL 수집
bio_essay_direct_urls = collect_list_urls(driver, bio_list_url)
print("============================================")

# 자기소개서 내용 크롤링
bio_results = collect_essays(driver, bio_essay_direct_urls)
print("============================================")

# JSON 파일로 저장
json_filename = "jobkorea_pass_essays_bio.json"
with open(json_filename, mode='w', encoding='utf-8-sig') as f:
    json.dump(bio_results, f, ensure_ascii=False, indent=2)

# 드라이버 종료
driver.quit()

In [ ]:
# 의료.바이오 분야 자기소개서 예시
bio_essay_direct_urls[0]
bio_results[0]

{'url': 'https://www.jobkorea.co.kr/starter/PassAssay/View/240736?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10044&Pass_An_Stat=0',
 'company': '(주)오뚜기',
 'title': '2022년 하반기 신입 바이오·제약연구원',
 'qas': [{'question': '지원직무를 위해 어떤 준비를 했는지 설명하십시오. (직무와 관련한 경험, 전공, 수강과목, 자격증 등을 포함하여 작성)',
   'answer': '학위과정 중 다양한 친환경 고분자 소재를 성형/가공하여 다양한 형태로 제조하였고, 이에 대한 물성 분석을 진행하였습니다.\n다양한 장비를 직접 운용하며 유변학적, 역학적, 열적, 광학적, 화학적, 전기적 특성을 측정하였고, 광범위한 분석 역량을 습득할 수 있었습니다.\n특히 “인열강도가 향상된 생분해성 PLA 필름 소재 개발”이란 국책과제에 참여하여 PLA, PBAT 필름의 역학적 강도 향상을 위해 다양한 나노 필러를 도입하였습니다. 유변 분석을 통해 고분자/필러 간 상호작용을 분석하였고, 이를 결정화 거동(XRD, DSC)/광 투과도(UV-vis)/열 안정성(TGA)/역학적 특성(UTM)을 분석하여 최종 목표인 인열 강도 향상을 위한 최적 레시피 도출에 성공하였습니다. 다양한 특성 간의 상관관계를 파악하는 역량을 강화하여 이를 바탕으로 주 저자로서 1편, 공동저자로 3편의 논문을 게재하였습니다.\n\n또한 셀룰로오스 나노섬유를 가스센서의 기판으로 도입하였습니다. 제조 방식에 따라 나노섬유의 기능기가 달랐음을 FT-IR을 통해 확인하였으며, Raman 분석을 통해 기판이 Oxy-SWCNTs에 도핑효과를 주었음을 확인하였습니다.\n이처럼 친환경 소재의 다양한 분석 경험과 이를 포장재에 응용하기 위한 레시피 도출 경험은 오뚜기에서도 친환경 포장재 개발에 큰 도움이 될 것이라 생각합니다.\n글자수 672자\n1,115Byte'},
  {'questio

## 마케팅.광고.MD 분야 자기소개서 수집

In [ ]:
# 잡코리아 로그인
driver = login()

# 자기소개서 목록 URL 수집
marketing_essay_direct_urls = collect_list_urls(driver, marketing_list_url)
print(marketing_essay_direct_urls)
print("============================================")

# 자기소개서 내용 크롤링
marketing_results = collect_essays(driver, marketing_essay_direct_urls)
print("============================================")

# JSON 파일로 저장
json_filename = "jobkorea_pass_essays_marketing.json"
with open(json_filename, mode='w', encoding='utf-8-sig') as f:
    json.dump(marketing_results, f, ensure_ascii=False, indent=2)

# 드라이버 종료
driver.quit()

In [ ]:
# 마케팅.광고.MD 분야 자기소개서 예시
marketing_essay_direct_urls[0]
marketing_results[0]

{'url': 'https://www.jobkorea.co.kr/starter/PassAssay/View/241746?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10030&Pass_An_Stat=0',
 'company': '이랜드월드',
 'title': '2023년 상반기 인턴 MD',
 'qas': [{'question': '삶을 통해 이루고 싶은 인생의 비전 또는 목표 3가지를 우선순위 순으로 적어주십시오. (300자)',
   'answer': '"전 세계를 향한 울림"\n이랜드월드에서 동료들과 함께 전 세계의 고객들이 열광하는 상품을 기획하고 싶습니다. 이를 위해 이커머스, 유통, 콘텐츠, 마케팅 등 다양한 직무에서 기획 역량을 쌓았습니다.\n\n"믿고 따를 수 있는 사람"\n좋은 사례 없이 성장하기란 쉽지 않습니다. 실무역량을 갖춘 멘토가 되어 후배들을 양성하고, 그들과 함께 미래 산업을 선도해나가고 싶습니다.\n\n"고객의 만족, 가족의 사랑"\n산업에 대한 관심을 바탕으로 고객들의 니즈를 만족시키는 데 최선을 다하며, 가족들에게도 절대 소홀하지 않은 사람이 되고 싶습니다.\n글자수 303자\n511Byte'},
  {'question': '자신이 다른 사람과 구별되는 능력이나 기질을 써주십시오. (300자)',
   'answer': '"책임감 아래 빠르게 배우려는 태도"\n두 달간 매일 새벽 6시 30분 OOO OOOO점에서 물류 입고도우미로 근무했습니다. 소비 트렌드와 유통업 전반에 대해 배우고 싶었기 때문입니다. 가장 가까운 곳에서 프로세스를 관찰하고 몸으로 느껴보자 다짐하며 업무에 임하였고, 하루 평균 300개 이상의 상품 박스를 나르고 배치하며 상권 특성에 따른 인기 상품의 종류와 특정 시즌을 준비하는 시기도 알 수 있었습니다. 비가 오는 날도 컨디션이\n나쁜 날도 있었지만, 책임감을 가지고 계약기간 내 한 번의 지각과 결근 없이 일한 결과였습니다.\n글자수 298자\n500Byte'},
  {'questi